In [4]:
import requests
from bs4 import BeautifulSoup
import urllib3
import pandas as pd
from tqdm import tqdm

In [ ]:
# URL to scrape
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
url = "https://thaicarbonlabel.tgo.or.th/index.php?lang=TH&mod=Y0hKdlpIVmpkSE5mWlcxcGMzTnBiMjQ9"

# Send a GET request
response = requests.get(url, verify=False)

# Check if the request was successful
if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'lxml')
    table = soup.find("table",{"class":"table table-striped table-bordered table-hover table-full-width table-customize"})
    dfs = pd.read_html(str(table))[0]
else:
    print(f"Failed to fetch page, status code: {response.status_code}")


In [ ]:
dfs

In [ ]:
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
url = "https://thaicarbonlabel.tgo.or.th/index.php?lang=TH&mod=YjNKbllXNXBlbUYwYVc5dVgyVnRhWE56YVc5dQ"


response = requests.get(url, verify=False)


if response.status_code == 200:
    soup = BeautifulSoup(response.content, 'lxml')
    table = soup.find("table",{"class":"table table-striped table-bordered table-hover table-full-width table-customize"})
    dfs = pd.read_html(str(table),header=[1,2])[0]
    dfs.columns = [" ".join(col).strip() if "unit" in col[1] else col[0] for col in dfs.columns]
else:
    print(f"Failed to fetch page, status code: {response.status_code}")


In [ ]:
dfs['Description'] = dfs['ลำดับ']
dfs['Description'] = dfs['Description'].fillna(dfs['ชื่อ'])
dfs.loc[1:23, 'Description'] = 'Stationary Combustion'
dfs.loc[24:31, 'Description'] = 'Mobile Combustion (On road)'
dfs.loc[31:37, 'Description'] = 'Mobile Combustion (Off road), Diesel'
dfs.loc[38:42, 'Description'] = 'Mobile Combustion (On road), Motor Gasoline 4 stroke'
dfs.loc[43:47, 'Description'] = 'Mobile Combustion (On road), Motor Gasoline 2 stroke'
dfs.loc[48, 'Description'] = 'Electricity, grid mix (ไฟฟ้า)'
dfs.loc[50:, 'Description'] = 'Refrigerants (สารทำความเย็น)'
dfs['ลำดับ'] = pd.to_numeric(dfs['ลำดับ'], errors='coerce')
dfs = dfs.dropna(subset=['ลำดับ'])
dfs = dfs.drop(columns=['CO2 [kg CO2/unit]','Fossil CH4 [kg CH4/unit]','CH4 [kg CH4/unit]','N2O [kg N2O/unit]'])
dfs

In [ ]:
approval-text-info
span9 approval-text

In [19]:
lst_license = []
lst_imgs_url,lst_img_details = [],[]
for num in tqdm(range(1,924)):
    urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
    # URL of the webpage
    url = f'https://thaicarbonlabel.tgo.or.th/index.php?lang=TH&mod=Y0hKdlpIVmpkSE5mWVhCd2NtOTJZV3c9&page={num}'
    response = requests.get(url, verify=False)
    soup = BeautifulSoup(response.content, 'html.parser')

    # License
    license_p = soup.find_all(class_='approval-text-info')
    for i, element in enumerate(license_p, start=1):
        h4_tags = element.find_all('h4')  # Find all <h4> inside the current element
        if h4_tags:
            for h4 in h4_tags:
                lst_license.append(h4.text)
        else:
            print(f"Section {i}: No <h4> found inside this element.")

    # Images
    approval_text_elements = soup.find_all(class_='span9 approval-text')
    for i, element in enumerate(approval_text_elements, start=1):
        img_tags = element.find_all('img')  # Find all <img> inside the current element
        if img_tags:
            for img in img_tags:
                img_src = img.get('src')
                img_alt = img.get('alt', 'No alt attribute')
                lst_imgs_url.append(img_src)
                lst_img_details.append(img_alt)
        else:
            print(f"Section {i}: No <img> found inside this element.")

100%|██████████| 923/923 [07:54<00:00,  1.95it/s]


In [32]:
df = pd.DataFrame({'License':lst_license,'img_URL':lst_imgs_url,'img_details':lst_img_details})
df.to_csv('url_imges.csv',index=False,encoding='utf-8-sig')
# print(len(lst_license),len(lst_imgs_url),len(lst_img_details))